In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
train_X, train_y = mnist.train.images, mnist.train.labels
test_X, test_y = mnist.test.images, mnist.test.labels

In [33]:
nn_input_dim = 784
nn_output_dim = 10
dropout_rate = 0.75
learning_rate = 0.001
training_epochs = 200000
batch_size = 128
display_step=10

def initialize_variables():
    x = tf.placeholder("float", shape=[None, nn_input_dim])
    y = tf.placeholder("float", shape=[None, nn_output_dim])
    keep_prob = tf.placeholder(tf.float32)
    
    return x, y, keep_prob


def initialize_weights():
    weights = {
        'w_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
        'w_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
        'w_full1': tf.Variable(tf.random_normal([7*7*64, 1024])),
        'w_out': tf.Variable(tf.random_normal([1024, nn_output_dim]))
    }
    
    biases = {
        'b_conv1': tf.Variable(tf.zeros([32])),
        'b_conv2': tf.Variable(tf.zeros([64])),
        'b_full1': tf.Variable(tf.zeros([1024])),
        'b_out': tf.Variable(tf.zeros([nn_output_dim]))
    }
    
    return weights, biases

In [28]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1, k, k, 1],
                          padding='SAME')
    
    
def ConvNet(x, weights, biases, dropout):
    #Reshape input image
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    #First Convolution Layer
    conv1 = conv2d(x, weights['w_conv1'], biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    #Second Convolution Layer
    conv2 = conv2d(conv1, weights['w_conv2'], biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    
    #Fully connected layer
    fc1 = tf.reshape(conv2, shape=[-1, weights['w_full1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['w_full1']), biases['b_full1'])
    fc1 = tf.nn.relu(fc1)
    
    #Apply dropout to reduce overfitting
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #Return prediction
    out = tf.add(tf.matmul(fc1, weights['w_out']), biases['b_out'])
    return out

In [29]:
x, y, keep_prob = initialize_variables()
w, b = initialize_weights()

#Make prediction
pred = ConvNet(x, w, b, keep_prob)

#Optimize the cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [34]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_epochs:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout_rate})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 24745.144531, Training Accuracy= 0.21875
Iter 2560, Minibatch Loss= 10358.794922, Training Accuracy= 0.53906
Iter 3840, Minibatch Loss= 3016.125977, Training Accuracy= 0.77344
Iter 5120, Minibatch Loss= 2787.668213, Training Accuracy= 0.81250
Iter 6400, Minibatch Loss= 3139.708496, Training Accuracy= 0.75781
Iter 7680, Minibatch Loss= 3405.146973, Training Accuracy= 0.83594
Iter 8960, Minibatch Loss= 3245.955078, Training Accuracy= 0.86719
Iter 10240, Minibatch Loss= 1670.103271, Training Accuracy= 0.89062
Iter 11520, Minibatch Loss= 1904.796143, Training Accuracy= 0.89062
Iter 12800, Minibatch Loss= 1292.114014, Training Accuracy= 0.89062
Iter 14080, Minibatch Loss= 1580.778809, Training Accuracy= 0.90625
Iter 15360, Minibatch Loss= 1843.487671, Training Accuracy= 0.89844
Iter 16640, Minibatch Loss= 1993.197021, Training Accuracy= 0.91406
Iter 17920, Minibatch Loss= 2380.957520, Training Accuracy= 0.89062
Iter 19200, Minibatch Loss= 1076.747925, Training Acc

Iter 157440, Minibatch Loss= 61.761642, Training Accuracy= 0.97656
Iter 158720, Minibatch Loss= 146.565582, Training Accuracy= 0.98438
Iter 160000, Minibatch Loss= 419.700592, Training Accuracy= 0.96094
Iter 161280, Minibatch Loss= 142.437820, Training Accuracy= 0.96875
Iter 162560, Minibatch Loss= 186.202820, Training Accuracy= 0.98438
Iter 163840, Minibatch Loss= 214.264618, Training Accuracy= 0.96875
Iter 165120, Minibatch Loss= 56.656628, Training Accuracy= 0.97656
Iter 166400, Minibatch Loss= 158.649292, Training Accuracy= 0.98438
Iter 167680, Minibatch Loss= 296.264587, Training Accuracy= 0.95312
Iter 168960, Minibatch Loss= 35.607872, Training Accuracy= 0.96875
Iter 170240, Minibatch Loss= 126.114738, Training Accuracy= 0.95312
Iter 171520, Minibatch Loss= 289.600281, Training Accuracy= 0.96094
Iter 172800, Minibatch Loss= 252.560242, Training Accuracy= 0.96875
Iter 174080, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 175360, Minibatch Loss= 55.644623, Training Accu